There are a lot of features that are correlated with the SalePrice.
Treat the NA as a category on it's own. If the feature is still important, it should show up.


Couple features we want to make sure.
MSSubClass -- appears to be number but it's actually categorical.

https://emredjan.github.io/blog/2017/07/11/emulating-r-plots-in-python/#leverageplot


#### 12 records within the test set are potentially problematics.
- They are index 1556, 1916, 1946, 2121, 2152, 2189, 2217, 2251, 2474, 2490, 2577, 2905 in the final combined set.
- They are Null in some fields which never existed in the train set.


In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


df_train = pd.read_csv('data/train.csv.gz', compression='gzip', header=0, sep=',', quotechar='"')
df_train_label = df_train[["Id", "SalePrice"]]
df_train = df_train.drop('SalePrice', axis=1)

df_train = df_train.set_index("Id")
df_train_label = df_train_label.set_index("Id")

df_test = pd.read_csv('data/test.csv.gz', compression='gzip', header=0, sep=',', quotechar='"')

test_null_columns=df_test.columns[df_test.isnull().any()] 
train_null_columns=df_train.columns[df_train.isnull().any()] 
test_null_only_ColIdx = test_null_columns.difference(train_null_columns)

test_null_only_RowIdx = [ df_test[df_test[idx].isnull()].index.tolist() for idx in test_null_only_ColIdx ]
test_null_only_RowIdx = list ( set(x for l in test_null_only_RowIdx for x in l) )

problematicTestSet = df_test.loc[ df_test.index.isin( test_null_only_RowIdx ) ]

fineTestSet = df_test.loc[ ~df_test.index.isin( test_null_only_RowIdx ) ]  #1447 records

problematicTestSet= problematicTestSet.set_index("Id")
fineTestSet = fineTestSet.set_index("Id")
df_test = df_test.set_index("Id")

df = pd.concat([df_train,df_test], axis=0, sort=True)

problematicTestSet.index


Int64Index([1556, 1916, 1946, 2121, 2152, 2189, 2217, 2251, 2474, 2490, 2577, 2905], dtype='int64', name='Id')

#### Define a function to check out the records in the fields that we will merge and dummy them together. 
- Correct Exterior1st and Exterior2nd, Replace  :Brk Cmn  -> BrkComm and CmentBd  -> CemntBd
- Check BsmtFinType1 and BsmtFinType2. Everything is fine.

In [2]:
def checkWeirdExtra (inputDF, field1, field2):
    var2_dummy_columns = pd.get_dummies(inputDF[field1], prefix= "x")
    #print( len( var2_dummy_columns.columns ) )

    var1_dummy_columns = pd.get_dummies(inputDF[field2], prefix= "x")
    #print( len( var1_dummy_columns.columns ) )

    var_dummy_columns = pd.concat([var1_dummy_columns,var2_dummy_columns],join='outer', sort=True).groupby(level=0).sum()
    #print( len( var_dummy_columns.columns ) )

    print ( set( var_dummy_columns.columns) - set( var1_dummy_columns.columns ) )
    print ( set( var_dummy_columns.columns) - set( var2_dummy_columns.columns ) )
    
    pass


checkWeirdExtra(df, "Exterior1st", "Exterior2nd")

df.Exterior1st = df.Exterior1st.str.replace("Brk Cmn", "BrkComm")
df.Exterior2nd = df.Exterior2nd.str.replace("Brk Cmn", "BrkComm")
df.Exterior1st = df.Exterior1st.str.replace("CmentBd", "CemntBd")
df.Exterior2nd = df.Exterior2nd.str.replace("CmentBd", "CemntBd")
df.Exterior1st = df.Exterior1st.str.replace("Wd Shng", "WdShing")
df.Exterior2nd = df.Exterior2nd.str.replace("Wd Shng", "WdShing")

checkWeirdExtra(df, "BsmtFinType1", "BsmtFinType2")


{'x_CemntBd', 'x_BrkComm', 'x_WdShing'}
{'x_CmentBd', 'x_Other', 'x_Brk Cmn', 'x_Wd Shng'}
set()
set()


#### Start to purposely encode the information based on our best understanding.
- Combine Exterior1st and Exterior2nd to **Exterior**
- BsmtFinType1 and BsmtFinType2 to **Bsmt** 
    -Replace each type to it's actually square feet  BsmtFinSF1, BsmtFinSF2
    -For Unf in type 1 and type2, replace it with the BsmtUnfSF
- Combine BsmtFullBath, BsmtHalfBath to **BsmtBath**
- Add all different PorchSF to **TotalProchSF**
- Dummy MasVnrType to **MasVnr** and replace the value with MasVnrArea
- Dummy PoolQC to **Pool** and replace the value with PoolArea

In [3]:
def purposelyEncodeData(inputDF):
    
    preProcessCatField = ["MasVnrType", "Exterior1st", "Exterior2nd", "BsmtFinType1", "BsmtFinType2"]
    preProcessNumFiled = ["MasVnrArea", "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "BsmtFullBath", "BsmtHalfBath"]
    
    inputDF[preProcessCatField] = inputDF[preProcessCatField].fillna("Unknown")
    inputDF[preProcessNumFiled] = inputDF[preProcessNumFiled].fillna(-1)
    
    # Need to encode some fields based on their definition and drop the original.
    # Exterior1st, Exterior2nd (Exterior covering on house)

    var1_dummy_columns = pd.get_dummies(inputDF['Exterior1st'], prefix= "Exterior")
    var2_dummy_columns = pd.get_dummies(inputDF['Exterior2nd'], prefix= "Exterior")
    var_dummy_columns = pd.concat([var1_dummy_columns,var2_dummy_columns], join='outer', sort=True).groupby(level=0).sum()
    var_dummy_columns = var_dummy_columns.replace(2, 1) 
    
    #for k, v in var1_dummy_columns.nunique().to_dict().items():
    #    print('{}={}'.format(k,v))

    inputDF = pd.concat([inputDF, var_dummy_columns], join='outer', sort=True, axis=1)
    inputDF = inputDF.drop( columns=['Exterior1st', 'Exterior2nd'] )
    print( inputDF.shape )

    # BsmtFinType1, BsmtFinType2, BsmtFinSF1 (Type 1 finished square feet), BsmtFinSF2 (Type 1 finished square feet), BsmtUnfSF: Unfinished square feet of basement area
    # TotalBsmtSF: Total square feet of basement area keep
    var1_dummy_columns = pd.get_dummies(inputDF['BsmtFinType1'], prefix= "Bsmt") 
    var1_dummy_columns = var1_dummy_columns.mul( inputDF['BsmtFinSF1'] , axis=0)
    tmp = var1_dummy_columns['Bsmt_Unf']
    tmp [ inputDF.loc[inputDF['BsmtFinType1'] == "Unf"].index ] = 1
    tmp = tmp.mul( inputDF['BsmtUnfSF'] , axis=0)
    var1_dummy_columns['Bsmt_Unf'] = tmp

    var2_dummy_columns = pd.get_dummies(inputDF['BsmtFinType2'], prefix= "Bsmt") 
    var2_dummy_columns = var2_dummy_columns.mul( inputDF['BsmtFinSF2'] , axis=0)
    tmp = var2_dummy_columns['Bsmt_Unf']
    tmp [ inputDF.loc[inputDF['BsmtFinType2'] == "Unf"].index ] = 1
    tmp = tmp.mul( inputDF['BsmtUnfSF'] , axis=0)
    var2_dummy_columns['Bsmt_Unf'] = tmp
    
    for i, v in var1_dummy_columns['Bsmt_Unf'].items():
        if v > 0:
            var2_dummy_columns['Bsmt_Unf'].loc[i] = 0

    var_dummy_columns = pd.concat([var1_dummy_columns,var2_dummy_columns], join='outer', sort=True).groupby(level=0).sum()

    #for k, v in var_dummy_columns.nunique().to_dict().items():
    #    print('{}={}'.format(k,v))

    inputDF = pd.concat([inputDF, var_dummy_columns], join='outer', sort=True, axis=1)
    inputDF = inputDF.drop( columns=['BsmtFinType1', 'BsmtFinType1', 'BsmtFinType2', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF'] )
    print( inputDF.shape )

    #BsmtFullBath, BsmtHalfBath  (number of type of bathroom in the basement)
    inputDF['BsmtBath'] = inputDF["BsmtFullBath"] + 0.5* inputDF["BsmtHalfBath"] 
    inputDF = inputDF.drop( columns=['BsmtFullBath', 'BsmtHalfBath'] )
    print( inputDF.shape )

    #OpenPorchSF ( Open porch area in square feet)
    #EnclosedPorch (Enclosed porch area in square feet)
    #3SsnPorch (Three season porch area in square feet)
    #ScreenPorch (Screen porch area in square feet)
    inputDF["TotalProchSF"] = inputDF["OpenPorchSF"] + inputDF["EnclosedPorch"] + inputDF["3SsnPorch"] + inputDF["ScreenPorch"] 
    print( inputDF.shape )

    #MasVnrType, MasVnrArea
    var_dummy_columns = pd.get_dummies(inputDF['MasVnrType'], prefix= "MasVnr") 
    var_dummy_columns = var_dummy_columns.mul( inputDF['MasVnrArea'] , axis=0)
    inputDF = pd.concat([inputDF, var_dummy_columns], join='outer', sort=True, axis=1)
    inputDF = inputDF.drop( columns=['MasVnrType', 'MasVnrArea'] )
    print( inputDF.shape )

    return (inputDF)


In [4]:
new = purposelyEncodeData(df.copy())
new.MSSubClass = new.MSSubClass.astype('object')
# new.OverallCond and new.OverallQual is already encoded. So, don't have to do anything.

(2919, 94)
(2919, 96)
(2919, 95)
(2919, 96)
(2919, 99)


In [5]:
new[ new["Bsmt_Unknown"] != 0 ].index
## index = 333, BsmtFinType2 is unknown but it has square feet for type2, we keep this as unknown b/c we don't know 
## index = 2121 Nan for all the BsmtFinType series.
## The cleaning should be techinically correct.
#df.columns


Int64Index([333, 2121], dtype='int64', name='Id')

#### Checking the level of each categorical variables

In [6]:
for c in new.columns:
    if new[c].dtype == 'object':
        print(c, len(new[c].value_counts()))

Alley 2
BldgType 5
BsmtCond 4
BsmtExposure 4
BsmtQual 4
CentralAir 2
Condition1 9
Condition2 8
Electrical 5
ExterCond 5
ExterQual 4
Fence 4
FireplaceQu 5
Foundation 6
Functional 7
GarageCond 5
GarageFinish 3
GarageQual 5
GarageType 6
Heating 6
HeatingQC 5
HouseStyle 8
KitchenQual 4
LandContour 4
LandSlope 3
LotConfig 5
LotShape 4
MSSubClass 16
MSZoning 5
MiscFeature 4
Neighborhood 25
PavedDrive 3
PoolQC 3
RoofMatl 8
RoofStyle 6
SaleCondition 6
SaleType 9
Street 2
Utilities 2


Ordinal Features
Label Count encoding is good in general, however, some of the features are ordinal in nature.
For example, we usually consider Excellent > Good > Average/Typical > Fair > Poor
We can construct a dictionary like the following and map it to those columns:

{'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa':2, 'Po':1}
You need a different dictionary for columns with different levels.

In [7]:
ord_cols = ['ExterQual', 'ExterCond','BsmtCond','HeatingQC', 'KitchenQual', 
           'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']
ord_dic = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa':2, 'Po':1}

ord_df = new.copy()
for col in ord_cols:
    ord_df[col] = ord_df[col].map(lambda x: ord_dic.get(x, 0))

In [8]:
new = ord_df.copy()

#### Label Count encoding or One-hot encoding ?
- Rank categorical variables by count in the **training** set and transform the test set
- Iterate counter for each CV fold - fit on the **new training set** and transform on the **new test set**
- Useful for both linear or non-linear algorithms

In [9]:
class LabelCountEncoder(object):
    def __init__(self):
        self.count_dict = {}
        self.rev_count_dict = {}
        
    def fit(self, column):
        # We want to rank the key by its value and use the rank as the new value
        count = column.value_counts()
        self.count_dict = dict( list( zip (count.index, reversed(range(len(count)+1 ) ) ) ) ) 
        self.rev_count_dict = dict( list( zip ( reversed(range(len(count)+1 ) ) , count.index ) ) ) 
            
    def transform(self, column):
        # If a category only appears in the test set, we will assign the value to zero.
        missing = 0
        return column.map(lambda x: self.count_dict.get(x, missing))
              
    def fit_transform(self, column):
        self.fit(column)
        return self.transform(column)

In [10]:
df_label_count = new.copy()
encodedDic = {}

for i, c in enumerate ( df_label_count.columns ):
    if df_label_count[c].dtype == 'object':
        lce = LabelCountEncoder()
        df_label_count[c] = lce.fit_transform(df_label_count[c])
        encodedDic[df_label_count.columns[i]] = lce.rev_count_dict
    else:
        df_label_count[c] = df_label_count[c].fillna(-1)
print( df_label_count.shape )


(2919, 99)


One-hot encode

In [11]:
df_onehot = new.copy()
object_feats = df_onehot.dtypes[df_onehot.dtypes == "object"].index
numeric_feats = df_onehot.dtypes[df_onehot.dtypes != "object"].index
objEnc = pd.get_dummies(df_onehot[object_feats], drop_first=True, dummy_na=True)
print( objEnc.shape )
numEnc = df_onehot[numeric_feats].fillna(-1)
print( numEnc.shape )
df_onehot = pd.concat( [objEnc,numEnc] , axis=1, join='outer', sort=True)
print( df_onehot.shape )
#df_onehot.sample(5)

(2919, 181)
(2919, 69)
(2919, 250)


In [12]:
df = df_label_count
#df = df_onehot
df_test = df[df.index >= min(df_test.index)]  ## determine if it is a test set or not. Have to encode all of them together.
df_train = df[df.index < min(df_test.index)]

Save the processed object to the pickle file

In [13]:
df_train.to_pickle("df_train.pkl")
df_train_label.to_pickle("df_train_label.pkl")
df_test.to_pickle("df_test.pkl")
df.to_pickle("df.pkl")


import pickle
with open('dic.pkl', 'wb') as handle:
    pickle.dump(encodedDic, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('dic.pkl', 'rb') as handle:
    savedEncodeDic = pickle.load(handle)

print (encodedDic == savedEncodeDic)


True


In [14]:
df.shape

(2919, 99)

In [15]:
x = encodedDic
x

{'Alley': {2: 'Grvl', 1: 'Pave'},
 'BldgType': {5: '1Fam', 4: 'TwnhsE', 3: 'Duplex', 2: 'Twnhs', 1: '2fmCon'},
 'BsmtExposure': {4: 'No', 3: 'Av', 2: 'Gd', 1: 'Mn'},
 'BsmtQual': {4: 'TA', 3: 'Gd', 2: 'Ex', 1: 'Fa'},
 'CentralAir': {2: 'Y', 1: 'N'},
 'Condition1': {9: 'Norm',
  8: 'Feedr',
  7: 'Artery',
  6: 'RRAn',
  5: 'PosN',
  4: 'RRAe',
  3: 'PosA',
  2: 'RRNn',
  1: 'RRNe'},
 'Condition2': {8: 'Norm',
  7: 'Feedr',
  6: 'Artery',
  5: 'PosN',
  4: 'PosA',
  3: 'RRNn',
  2: 'RRAe',
  1: 'RRAn'},
 'Electrical': {5: 'SBrkr', 4: 'FuseA', 3: 'FuseF', 2: 'FuseP', 1: 'Mix'},
 'Fence': {4: 'MnPrv', 3: 'GdPrv', 2: 'GdWo', 1: 'MnWw'},
 'Foundation': {6: 'PConc',
  5: 'CBlock',
  4: 'BrkTil',
  3: 'Slab',
  2: 'Stone',
  1: 'Wood'},
 'Functional': {7: 'Typ',
  6: 'Min2',
  5: 'Min1',
  4: 'Mod',
  3: 'Maj1',
  2: 'Maj2',
  1: 'Sev'},
 'GarageFinish': {3: 'Unf', 2: 'RFn', 1: 'Fin'},
 'GarageType': {6: 'Attchd',
  5: 'Detchd',
  4: 'BuiltIn',
  3: 'Basment',
  2: '2Types',
  1: 'CarPort'},
 